# Predictions by Llama2 for IBL

<a href="https://colab.research.google.com/github/fuyu-quant/ibl-qlora/blob/main/examples/ibl_llama2_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 以下のリンクからモデルを選んでください(モデルとデータセットはついになっています)
*  https://huggingface.co/fuyu-quant

In [1]:
%%capture
!pip install bitsandbytes==0.40.0 transformers==4.31.0 peft==0.4.0 accelerate==0.21.0 sentencepiece==0.1.99 einops==0.6.1 datasets
!pip install --upgrade ibltuning

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch
from peft import PeftModel
from peft import AutoPeftModelForCausalLM

from ibltuning import iblboosting

from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('huggingface'))

#dataset_name = 'ibl-regression-ver2-branch_easy'
#dataset_name = 'ibl-regression-ver2-linear'
dataset_name = 'ibl-regression-ver2-all'
model_name = f'fuyu-quant/{dataset_name}'


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122_nocublaslt.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122_nocublaslt.so...
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-13b-hf')
model = AutoPeftModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

## テスト用データのダウンロード

In [4]:
test_dataset = load_dataset(f"fuyu-quant/{dataset_name}", split='test')

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## IBLブースティング

In [10]:
i = 10
num_boost_round = 10


true_data = test_dataset[i]['output']
print('正解データ:\n', true_data)

print('------')
input_data = test_dataset[i]['instruction']
code_list = iblboosting.run(model, tokenizer, input_data, num_boost_round)
print('出力データ:\n', code_list)

正解データ:
 y = 4*row["A"] + 7*row["B"] + 7*row["C"]
------
出力データ:
 ['y = 4*row["A"] + 5*row["B"] + 2*row["C"]', 'y = 0*row["A"] + 2*row["B"] + 0*row["C"]', 'y = 0*row["A"] + 0*row["B"] + 5*row["C"]', 'if row["C"] < 20:\n    y = 0\nelse:\n    y = 1', 'if row["C"] < 20:\n    y = 0\nelif row["B"] < 21:\n    y = -1\nelse:\n    y = 1', 'if row["B"] < 20:\n    y = 0\nelse:\n    y = 1', 'if row["C"] < 21:\n    y = 0\nelif row["C"] < 19:\n    y = 2\nelif row["C"] < 11:\n    y = 1\nelse:\n    y = -1', 'if row["B"] < 21:\n    y = 0\nelse:\n    y = -1', 'if row["A"] < 21:\n    y = 0\nelse:\n    y = -1', 'if row["C"] < 21:\n    y = 0\nelif row["B"] < 20:\n    y = -1\nelse:\n    y = 0']


In [11]:
code_list

['y = 4*row["A"] + 5*row["B"] + 2*row["C"]',
 'y = 0*row["A"] + 2*row["B"] + 0*row["C"]',
 'y = 0*row["A"] + 0*row["B"] + 5*row["C"]',
 'if row["C"] < 20:\n    y = 0\nelse:\n    y = 1',
 'if row["C"] < 20:\n    y = 0\nelif row["B"] < 21:\n    y = -1\nelse:\n    y = 1',
 'if row["B"] < 20:\n    y = 0\nelse:\n    y = 1',
 'if row["C"] < 21:\n    y = 0\nelif row["C"] < 19:\n    y = 2\nelif row["C"] < 11:\n    y = 1\nelse:\n    y = -1',
 'if row["B"] < 21:\n    y = 0\nelse:\n    y = -1',
 'if row["A"] < 21:\n    y = 0\nelse:\n    y = -1',
 'if row["C"] < 21:\n    y = 0\nelif row["B"] < 20:\n    y = -1\nelse:\n    y = 0']

## テスト実行

In [ ]:
def ibl_output(model, input):
    prompt = f"### Instruction: {input}\n\n### Response: "

    # 推論の実行
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100
            )
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = output.split("### Response:  ")[-1]
    return output

In [ ]:
i = 13

input_data = test_dataset[i]['instruction']
output_data = ibl_output(model, input_data)
print('出力データ:\n', output_data)

print('------')
true_data = test_dataset[i]['output']
print('正解データ:\n', true_data)